In [1]:
import numpy as np
from dipy.io.streamline import load_tractogram
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_fnames
from dipy.viz import window, actor, colormap

/home/ang/anaconda3/envs/SupWMA/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fname = get_fnames('fornix')

In [3]:
print(fname)

/home/ang/anaconda3/envs/SupWMA/lib/python3.6/site-packages/dipy/data/files/tracks300.trk


In [4]:
fornix = load_tractogram(fname, 'same', bbox_valid_check=False)
streamlines = fornix.streamlines

In [5]:
len(streamlines) # 300 streamlines in fornix
# stream_numpy = np.ndarray(streamlines)
len(streamlines[0]) # No of points per streamlines (points per streamline vary a lot)

79

In [6]:
qb = QuickBundles(threshold=10.)
clusters = qb.cluster(streamlines)

In [7]:
print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)
# TODO: why are there 12 points to one centroid of a cluster

# this centroid, is 12 points downsampled from original streamline which depicts the 
# overall bundle representation.

Nb. clusters: 4
Cluster sizes: <map object at 0x7f727405f320>
Small clusters: [False False False  True]
Streamlines indices of the first cluster:
 [0, 7, 8, 10, 11, 12, 13, 14, 15, 18, 26, 30, 33, 35, 41, 65, 66, 85, 100, 101, 105, 115, 116, 119, 122, 123, 124, 125, 126, 128, 129, 135, 139, 142, 143, 144, 148, 151, 159, 167, 175, 180, 181, 185, 200, 208, 210, 224, 237, 246, 249, 251, 256, 267, 270, 280, 284, 293, 296, 297, 299]
Centroid of the last cluster:
 [[ 84.83774  117.9259    77.322784]
 [ 86.108505 115.84363   81.91885 ]
 [ 86.40357  112.25677   85.7293  ]
 [ 86.48337  107.60328   88.137825]
 [ 86.238976 102.51007   89.29447 ]
 [ 85.04564   97.460205  88.542404]
 [ 82.6024    93.14851   86.84209 ]
 [ 78.98937   89.57682   85.63652 ]
 [ 74.72344   86.60828   84.939186]
 [ 70.40846   85.158745  82.4484  ]
 [ 66.745346  86.002625  78.82582 ]
 [ 64.02451   88.43942   75.06974 ]]


In [31]:
# Enables/disables interactive visualization
interactive = False

scene = window.Scene()
scene.SetBackground(1, 1, 1)
scene.add(actor.streamtube(streamlines, window.colors.white))
window.record(scene, out_path='fornix_initial.png', size=(1000, 1000))
if interactive:
    window.show(scene)

In [32]:
colormap = colormap.create_colormap(np.arange(len(clusters)))

scene.clear()
scene.SetBackground(1, 1, 1)
scene.add(actor.streamtube(streamlines, window.colors.white, opacity=0.05))
scene.add(actor.streamtube(clusters.centroids, colormap, linewidth=0.4))
window.record(scene, out_path='fornix_centroids.png', size=(600, 600))
if interactive:
    window.show(scene)

scene.clear()

In [33]:
colormap_full = np.ones((len(streamlines), 3))
for cluster, color in zip(clusters, colormap):
    colormap_full[cluster.indices] = color

scene.clear()
scene.SetBackground(1, 1, 1)
scene.add(actor.streamtube(streamlines, colormap_full))
window.record(scene, out_path='fornix_clusters.png', size=(600, 600))
if interactive:
    window.show(scene)